## Integrate SPI into countries and regions (AOI section)
### This notebook adds SPI values to existing precalculated tables for countries and regions. It also updates the field "percentage_protected" to the September 2023 version of the WDPAs.

In [1]:
# Import libraries
import geopandas as gpd
import pandas as pd

In [2]:
# Set path
path = '/Users/sofia/Documents/HE_Data'

In [3]:
# Read precalculated tables for countries and regions (used in AOI section of the H-E Map)
gadm1 = gpd.read_file(path + '/Precalculated/gadm1/Outputs/gadm1_precalculated_aoi_summaries_updated.geojson')
gadm0 = gpd.read_file(path + '/Precalculated/gadm0/Outputs/gadm0_precalculated_aoi_summaries_updated.geojson')

In [4]:
# Read the .csv files for SPI (sent by MOL)
gadm0_spi = pd.read_csv(path + '/SPI_AOI/Terrestrial_SPI_by_country_20231002_withiso.csv')
gadm1_spi = pd.read_csv(path + '/SPI_AOI/Terrestrial_SPI_by_region_20231002_withiso.csv')

There are missing iso codes in the SPI datasets. 

In [ ]:
# # Save a list of the missing iso codes to a .txt file
# folder_path = path + "/SPI_AOI/"
# file_path = folder_path + "missing_countries.txt"

# with open(file_path, "w") as file:
#     for item in a:
#         file.write(item + "\n")
        

In [ ]:
# # Save a list of the missing iso codes to a .txt file
# folder_path = path + "/SPI_AOI/"
# file_path = folder_path + "missing_countries.txt"

# with open(file_path, "w") as file:
#     for item in a:
#         file.write(item + "\n")

### Join SPI and protection information to AOI precalculated tables

Add info for missing countries and regions (provided by MOL separately)

In [5]:
# Danyan (MOL): "only 1 out of 26 country has different GID and ISO code, all the others that are missing do not have SPI calculated from our end"
# Replace 'ZNC' with 'XNC' in the GID_0 column of gadm0_spi to match the GID_0 column of gadm0
gadm0_spi.loc[gadm0_spi['GID_0'] == 'ZNC', 'GID_0'] = 'XNC'

In [7]:
# For missing regions MOL has provided a lookup table. 
missing = pd.read_csv(path + '/SPI_AOI/spi_regions_missing_lookup_20231006.csv')
missing = missing[['SPI_high', 'percentprotected_high', 'ISO3_region']].rename(columns={'ISO3_region': 'GID_1'})
missing.head()

,SPI_high,percentprotected_high,GID_1
0,6.95,1.21,ALA.1_1
1,6.95,1.21,ALA.4_1
2,6.95,1.21,ALA.8_1
3,6.95,1.21,ALA.9_1
4,6.95,1.21,ALA.14_1


In [8]:
# Get 2023 for the 'refine' mode
gadm1_2023 = gadm1_spi[(gadm1_spi['year'] == 2023) & (gadm1_spi['mode'] == 'Refine')]
gadm0_2023 = gadm0_spi[(gadm0_spi['year'] == 2023) & (gadm1_spi['mode'] == 'Refine')]

/var/folders/98/0pdnjc5s29x2pnzl293pw7hr0000gn/T/ipykernel_2163/3543887495.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gadm0_2023 = gadm0_spi[(gadm0_spi['year'] == 2023) & (gadm1_spi['mode'] == 'Refine')]


In [9]:
# Keep only relevant fields
gadm0_2023 = gadm0_2023[['GID_0', 'SPI_high', 'percentprotected_high']]
gadm1_2023 = gadm1_2023[['GID_1', 'SPI_high', 'percentprotected_high']]

In [10]:
# Concatenate missing and gadm1_2023
gadm1_2023 = pd.concat([gadm1_2023, missing], ignore_index=True)
gadm1_2023.head()

,GID_1,SPI_high,percentprotected_high
0,AFG.1_1,48.39,20.70
1,AFG.2_1,0.00,0.00
2,AFG.3_1,12.06,2.39
3,AFG.4_1,0.00,0.00
4,AFG.5_1,87.78,30.51


In [11]:
# Update precalculated tables adding SPI and new protection for 2023
gadm0_updated = pd.merge(gadm0, gadm0_2023 ,how='left', left_on=['GID_0'], right_on = ['GID_0'])
gadm1_updated = pd.merge(gadm1, gadm1_2023 ,how='left', left_on=['GID_1'], right_on = ['GID_1'])

gadm0_updated['percentage_protected'] = gadm0_updated['percentprotected_high']
gadm1_updated['percentage_protected'] = gadm1_updated['percentprotected_high']

gadm0_updated = gadm0_updated.rename(columns={'SPI_high': 'SPI'}).drop(columns=['percentprotected_high'])
gadm1_updated = gadm1_updated.rename(columns={'SPI_high': 'SPI'}).drop(columns=['percentprotected_high'])

In [14]:
gadm0_updated[gadm0_updated['GID_0'] == 'XNC']

,GID_0,NAME_0,MOL_ID,AREA_KM2,alternative_names,NAME_0_es,NAME_0_fr,NAME_0_pt,amphibians,birds,...,land_cover_majority,climate_regime_majority,country_size,agriculture,builtup,extraction,intrusion,transportation,geometry,SPI
248,XNC,Northern Cyprus,249,3313.962437,None,Chipre del Norte,Chypre du Nord,Chipre do Norte,"[ { ""SliceNumber"": 955, ""per_global"": 0.11, ""S...","[ { ""SliceNumber"": 121, ""per_global"": 0.0, ""SP...",...,Cropland,Sub Tropical Dry,5,"[ { ""Year"": 1990, ""percentage_land_encroachmen...","[ { ""Year"": 1990, ""percentage_land_encroachmen...",None,"[ { ""Year"": 1990, ""percentage_land_encroachmen...","[ { ""Year"": 1990, ""percentage_land_encroachmen...","MULTIPOLYGON (((32.60200 35.17820, 32.62292 35...",3.86


In [20]:
# Save to geojson
gadm0_updated.to_file(path + '/Precalculated/gadm0/Outputs/gadm0_precalculated_aoi_summaries_SPI.geojson',driver='GeoJSON')
gadm1_updated.to_file(path + '/Precalculated/gadm1/Outputs/gadm1_precalculated_aoi_summaries_SPI.geojson',driver='GeoJSON')